<h1 style="color:cyan">Scrapping Data</h1>

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
# Set up WebDriver (No need for ChromeDriver path)
driver = webdriver.Chrome()


# Function to log in to Spotify
def login_spotify():
    driver.get("https://accounts.spotify.com/en-GB/login?continue=https%3A%2F%2Fopen.spotify.com%2F")
    time.sleep(3)

    # Locate username, password fields, and login button
    email_input = driver.find_element(By.ID, "login-username")
    password_input = driver.find_element(By.ID, "login-password")
    login_button = driver.find_element(By.ID, "login-button")

    # Enter login details
    email_input.send_keys("dsmayanksinghrana.neubrain@gmail.com")
    password_input.send_keys("dsmayank@26")
    login_button.click()

    time.sleep(5)  # Wait for login to complete

# List of Spotify chart URLs for January
urls = [
    "https://charts.spotify.com/charts/view/regional-in-weekly/2025-01-02",
    "https://charts.spotify.com/charts/view/regional-in-weekly/2025-01-09",
    "https://charts.spotify.com/charts/view/regional-in-weekly/2025-01-16",
    "https://charts.spotify.com/charts/view/regional-in-weekly/2025-01-23",
    "https://charts.spotify.com/charts/view/regional-in-weekly/2025-01-30"
]

# Log in to Spotify
login_spotify()

all_data = []

for url in urls:
    driver.get(url)
    time.sleep(5)  

    # Extract table rows
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        try:
            rank = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text
            track = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text
            peak = row.find_element(By.CSS_SELECTOR, "td:nth-child(4)").text
            prev = row.find_element(By.CSS_SELECTOR, "td:nth-child(5)").text
            streak = row.find_element(By.CSS_SELECTOR, "td:nth-child(6)").text
            streams = row.find_element(By.CSS_SELECTOR, "td:nth-child(7)").text
            
            all_data.append({
                "Date": url.split("/")[-1],  # Extract date from URL
                "Rank": rank,
                "TRACK": track,
                "Peak": peak,
                "Prev": prev,
                "Streak": streak,
                "Streams": streams
            })
        except:
            continue

driver.quit()

# Convert to DataFrame and save as CSV
df = pd.DataFrame(all_data)
df.to_csv("spotify.csv", index=False)



In [2]:
data = pd.read_csv("spotify.csv")

<h1 style="color:cyan">Data Cleaning</h1>

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     1000 non-null   object
 1   Rank     1000 non-null   object
 2   TRACK    1000 non-null   object
 3   Peak     1000 non-null   int64 
 4   Prev     1000 non-null   object
 5   Streak   1000 non-null   int64 
 6   Streams  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [12]:
# converting the data column to datetime formate
data['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [13]:
data.head()

,Date,Rank,TRACK,Peak,Prev,Streak,Streams
0,2025-01-02,1\n –,Jo Tum Mere Ho\nAnuv Jain,1,1,21,"6,734,459"
1,2025-01-02,2\n –,"Ishq Hai\nAnurag Saikia, Raj Shekhar, Romy, Am...",2,2,3,"6,681,446"
2,2025-01-02,3\n –,"Raanjhan (From ""Do Patti"")\nSachet-Parampara, ...",2,3,11,"5,670,328"
3,2025-01-02,4\n –,"Die With A Smile\nLady Gaga, Bruno Mars",4,4,14,"5,480,132"
4,2025-01-02,5\n1,"Sahiba\nJasleen Royal, Stebin Ben, Vijay Dever...",5,6,6,"5,120,142"


In [14]:
data.Rank

0             1\n – 
1             2\n – 
2             3\n – 
3             4\n – 
4               5\n1
           ...      
995         196\nNew
996          197\n35
997    198\nRe-Entry
998          199\n27
999          200\n13
Name: Rank, Length: 1000, dtype: object

In [15]:
# cleaning the uneven rank data 
data['Rank'] = data['Rank'].str.split('\n').str[0]

In [16]:
data.head()

,Date,Rank,TRACK,Peak,Prev,Streak,Streams
0,2025-01-02,1,Jo Tum Mere Ho\nAnuv Jain,1,1,21,"6,734,459"
1,2025-01-02,2,"Ishq Hai\nAnurag Saikia, Raj Shekhar, Romy, Am...",2,2,3,"6,681,446"
2,2025-01-02,3,"Raanjhan (From ""Do Patti"")\nSachet-Parampara, ...",2,3,11,"5,670,328"
3,2025-01-02,4,"Die With A Smile\nLady Gaga, Bruno Mars",4,4,14,"5,480,132"
4,2025-01-02,5,"Sahiba\nJasleen Royal, Stebin Ben, Vijay Dever...",5,6,6,"5,120,142"


In [17]:
# convert the rank column into integer for futher analysis
data['Rank'] = pd.to_numeric(data['Rank'], errors='coerce')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1000 non-null   datetime64[ns]
 1   Rank     1000 non-null   int64         
 2   TRACK    1000 non-null   object        
 3   Peak     1000 non-null   int64         
 4   Prev     1000 non-null   object        
 5   Streak   1000 non-null   int64         
 6   Streams  1000 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 54.8+ KB


In [19]:
# Replacing the "\n" charecters with space " "
data["TRACK"] = data["TRACK"].str.replace("\n", " ")

In [20]:
data

,Date,Rank,TRACK,Peak,Prev,Streak,Streams
0,2025-01-02,1,Jo Tum Mere Ho Anuv Jain,1,1,21,"6,734,459"
1,2025-01-02,2,"Ishq Hai Anurag Saikia, Raj Shekhar, Romy, Ama...",2,2,3,"6,681,446"
2,2025-01-02,3,"Raanjhan (From ""Do Patti"") Sachet-Parampara, P...",2,3,11,"5,670,328"
3,2025-01-02,4,"Die With A Smile Lady Gaga, Bruno Mars",4,4,14,"5,480,132"
4,2025-01-02,5,"Sahiba Jasleen Royal, Stebin Ben, Vijay Devera...",5,6,6,"5,120,142"
...,...,...,...,...,...,...,...
995,2025-01-30,196,"Bol Na Halke Halke Shankar-Ehsaan-Loy, Rahat F...",196,—,1,"1,272,561"
996,2025-01-30,197,"Hey Minnale G. V. Prakash, Haricharan, Shweta ...",10,162,16,"1,264,941"
997,2025-01-30,198,"Khairiyat Pritam, Arijit Singh",1,—,1,"1,256,738"
998,2025-01-30,199,Lemonade Diljit Dosanjh,26,172,126,"1,249,160"


In [21]:
# convert the Peak column into integer for futher analysis
data['Peak'] = pd.to_numeric(data['Peak'], errors='coerce')

In [22]:
# convert the Prev column into integer for futher analysis
data['Prev'] = pd.to_numeric(data['Prev'], errors='coerce')

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1000 non-null   datetime64[ns]
 1   Rank     1000 non-null   int64         
 2   TRACK    1000 non-null   object        
 3   Peak     1000 non-null   int64         
 4   Prev     948 non-null    float64       
 5   Streak   1000 non-null   int64         
 6   Streams  1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 54.8+ KB


In [24]:
# convert the Streak column into integer for futher analysis
data['Streak'] = pd.to_numeric(data['Streak'], errors='coerce')

In [25]:
# convert the Streak column into integer for futher analysis
#data['Streams'] = pd.to_numeric(data['Streams'], errors='coerce')

In [26]:
# replacing the "," with nospace
data['Streams'] = data['Streams'].str.replace(',', '', regex=False)

In [27]:
## convert the Streak column into integer for futher analysis
data['Streams'] = pd.to_numeric(data['Streams'], errors='coerce')

In [28]:
## checking the null values of Prev
data.Prev.isnull().sum()


52

In [29]:
# filling the null values of preve with 0
data['Prev'] = df['Prev'].fillna(0)

In [30]:
data.Prev.isnull().sum()

0

In [31]:
data.head()

,Date,Rank,TRACK,Peak,Prev,Streak,Streams
0,2025-01-02,1,Jo Tum Mere Ho Anuv Jain,1,1,21,6734459
1,2025-01-02,2,"Ishq Hai Anurag Saikia, Raj Shekhar, Romy, Ama...",2,2,3,6681446
2,2025-01-02,3,"Raanjhan (From ""Do Patti"") Sachet-Parampara, P...",2,3,11,5670328
3,2025-01-02,4,"Die With A Smile Lady Gaga, Bruno Mars",4,4,14,5480132
4,2025-01-02,5,"Sahiba Jasleen Royal, Stebin Ben, Vijay Devera...",5,6,6,5120142


In [32]:
# save the cleaned data to csv format
data.to_csv("clean_spotify.csv", header=True, index=False)

In [33]:
data.head(50)

,Date,Rank,TRACK,Peak,Prev,Streak,Streams
0,2025-01-02,1,Jo Tum Mere Ho Anuv Jain,1,1,21,6734459
1,2025-01-02,2,"Ishq Hai Anurag Saikia, Raj Shekhar, Romy, Ama...",2,2,3,6681446
2,2025-01-02,3,"Raanjhan (From ""Do Patti"") Sachet-Parampara, P...",2,3,11,5670328
3,2025-01-02,4,"Die With A Smile Lady Gaga, Bruno Mars",4,4,14,5480132
4,2025-01-02,5,"Sahiba Jasleen Royal, Stebin Ben, Vijay Devera...",5,6,6,5120142
5,2025-01-02,6,"Ishq Faheem Abdullah, Rauhan Malik, Amir Ameer",2,5,40,4720228
6,2025-01-02,7,Wavy Karan Aujla,4,7,7,4526435
7,2025-01-02,8,"Aaj Ki Raat (From ""Stree 2"") Sachin-Jigar, Mad...",1,8,23,4404628
8,2025-01-02,9,Millionaire Yo Yo Honey Singh,3,10,18,4270616
9,2025-01-02,10,"Sajni (From ""Laapataa Ladies"") Ram Sampath, Ar...",1,9,35,3850295
